In [4]:
import os.path
from pathlib import Path
import time
from tqdm import tqdm
import csv
from PIL import Image
import math
import torch
import ast

from Embedding_Classes import EmbeddingResNet
from Database_Classes import DatabaseGeopandasPolygons

import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torch.utils.data import Dataset
import os
import matplotlib.pyplot as plt
import pandas as pd 
import torch.optim as optim
import ast
import csv
import time
from tqdm import tqdm
from tabulate import tabulate

# Filter out UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
database = DatabaseGeopandasPolygons()

neighbourhood_images, images, neighbourhoods = database.run()
print('loaded data')
embedder = EmbeddingResNet()

wanted_hoods = [i for i in range(1, 2, 1)]
image_paths = []
embeddings = []
# print('embedding')
# start_time = time.time()

image_ids = []
images_to_skip = [22607, 22626, 22630, 25076, 25996, 26001, 26018]

loaded data


In [3]:
for index, row in tqdm(images.iterrows(), total=len(images)):
    if row['img_id'] in images_to_skip:
        continue

    image_ids.append(str(row['img_id_com']))
    image_paths.append('U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/' + row['path'])

100%|██████████| 143424/143424 [00:06<00:00, 22599.53it/s]


In [11]:
version_split = str(1.0).split('.')
version_str = f'v{version_split[0]}_{version_split[1]}'

folder_path = 'C:/Users/Soulless Creature/Dropbox/WB3/Q2/Capstone Project/Repositories/Capstone-Visual-Summary/Embedding Files'

files = [folder_path + '/' + f for f in os.listdir(folder_path) if version_str in f]
files = sorted(files, key=lambda x: int(x.split('_')[3].split('_')[0]))

v_1_0_embeddings = dict()

for file in tqdm(files, total=len(files)):
    with open(file, mode='r', newline='', encoding='utf-8') as csvfile:
        temp = list(csv.DictReader(csvfile, delimiter=';'))

        for row in temp:
            v_1_0_embeddings[row['image_id']] = torch.Tensor(ast.literal_eval(row['tensor']))


100%|██████████| 60/60 [12:14<00:00, 12.24s/it]


In [5]:
class FineTuneResNet152(nn.Module):
    def __init__(self, feature_size, output_size):
        super(FineTuneResNet152, self).__init__()
        # Here we only use the fully connected layers for fine-tuning
        self.fc = nn.Linear(feature_size, output_size)

    def forward(self, x):
        # Directly use the embeddings for fine-tuning
        x = self.fc(x)
        return x

In [7]:
model = FineTuneResNet152(feature_size=2048, output_size=2048).to(device)
model.load_state_dict(torch.load('resnet152_trained_triplet_epoch2_num_triplets20.pt'))

<All keys matched successfully>

In [16]:
batch_size = 500
for j in range(60):
    # start = 100408 + j * 14344
    # length = min(114752 + j * 14344, len(image_paths))
    start = 0 + j * 2392
    length = min(2392 + j * 2392, len(image_paths))
    for i in tqdm(range(start, length, batch_size)):
        embedder = EmbeddingResNet()
        end_index = min(i + batch_size, length)
        
        embeddings_dict = dict()
        
        for image_id in image_ids[i:end_index]:
            embeddings_dict[image_id] = model(v_1_0_embeddings[image_id].to('cuda'))
    
        # embeddings_dict = dict()
        # 
        # counter = 0
        # for index in range(i, end_index):
        #     embeddings_dict[image_ids[index]] = embeddings[counter].squeeze()
        #     counter += 1
        
        file_name = f'Embedding Files/Embeddings_v2_0_{start}_{length - 1}.csv'
    
        with open(file_name, mode='a+', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.DictWriter(csvfile, fieldnames=['image_id', 'tensor'], delimiter=';')
    
            if csvfile.tell() == 0:
                csv_writer.writeheader()
    
            for image_id, tensor in embeddings_dict.items():
                csv_writer.writerow({'image_id': image_id, 'tensor': tensor.tolist()})

100%|██████████| 5/5 [00:04<00:00,  1.02it/s]


In [7]:
image_embeddings = dict()
start_time = time.time()
with open('Embedding Files/Embeddings_1_0_0.csv', mode='r', newline='') as csvfile:
    temp = csv.DictReader(csvfile, delimiter=';')

    for row in temp:
        image_embeddings[row['image_id']] = torch.Tensor(ast.literal_eval(row['tensor']))
        # image_embeddings[row['image_id']] = row['tensor']

end_time = time.time()
print(f'Time taken: {end_time - start_time}s')

Time taken: 0.04166293144226074s


In [12]:
for key, value in image_embeddings.items():
    print(f'Read: {value}')
    print(f'Write: {value}')
    if not torch.equal(value, embeddings_dict[key]):
        print(False)
    

Read: tensor([2.1685, 0.7097, 1.0430,  ..., 0.1948, 0.8476, 0.1679])
Write: tensor([2.1685, 0.7097, 1.0430,  ..., 0.1948, 0.8476, 0.1679])
Read: tensor([1.9091, 0.1310, 0.3616,  ..., 0.3909, 0.0516, 0.2608])
Write: tensor([1.9091, 0.1310, 0.3616,  ..., 0.3909, 0.0516, 0.2608])
Read: tensor([1.5261, 0.3460, 0.6064,  ..., 0.6296, 0.3954, 0.2984])
Write: tensor([1.5261, 0.3460, 0.6064,  ..., 0.6296, 0.3954, 0.2984])
Read: tensor([1.2783, 0.7537, 0.7907,  ..., 0.2330, 0.6333, 0.1101])
Write: tensor([1.2783, 0.7537, 0.7907,  ..., 0.2330, 0.6333, 0.1101])
Read: tensor([1.3602, 0.4002, 1.6330,  ..., 0.2130, 0.5594, 0.3256])
Write: tensor([1.3602, 0.4002, 1.6330,  ..., 0.2130, 0.5594, 0.3256])


In [28]:
missing_embeddings = [100408, 100409, 100410, 100411, 100412, 100413, 100414, 100415, 100416, 100417, 100418, 100419, 100420, 100421, 100422, 100423]
missing_embeddings_path = []

for missing_embedding in missing_embeddings:
    missing_embeddings_path.append('U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/' + images.loc[(images['img_id_com'] == missing_embedding), 'path'].iloc[0])

embedder = EmbeddingResNet()

embeddings = embedder.run(paths=missing_embeddings_path)

embeddings_dict = dict()

counter = 0
for index in range(len(missing_embeddings)):
    embeddings_dict[missing_embeddings[index]] = embeddings[counter].squeeze()
    counter += 1

file_name = f'Embedding Files/Missing_Embeddings.csv'

with open(file_name, mode='a+', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.DictWriter(csvfile, fieldnames=['image_id', 'tensor'], delimiter=';')

    if csvfile.tell() == 0:
        csv_writer.writeheader()

    for image_id, tensor in embeddings_dict.items():
        csv_writer.writerow({'image_id': image_id, 'tensor': tensor.tolist()})

In [27]:
embedder = EmbeddingResNet()

print(embedder.run(paths=['U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/image_25102_s_a.png'])[0].squeeze())
print(embedder.run(paths=['U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/image_25106_s_a.png'])[0].squeeze())
print(embedder.run(paths=['U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/image_35855_b.png'])[0].squeeze())

tensor([1.0861, 0.2147, 0.0970,  ..., 0.8504, 0.1070, 0.4913])
tensor([1.7512, 0.2217, 0.6170,  ..., 0.6139, 0.0566, 0.2008])
tensor([1.6515, 0.0395, 1.0881,  ..., 0.1610, 0.2274, 0.0518])
